In [1]:
import sys
sys.path.append("..")

from lib.dataset_wrapper import Dataset
import matplotlib.pyplot as plt
import ipywidgets as ipw
import numpy as np

from scipy import signal

datasets_name = ["fsew0", "msak0", "pb2007"]

In [2]:
EMA_LOWPASS_CUTOFF = 50
FILES_LOWPASS_CUTOFF = 10

In [3]:
# function sig_out = low_pass(sig_in, fech, fcc, R)
#     if ~exist('R', 'var') R = 0.15; end
#     fc =  fcc*2/fech; % end of passband
#     fc1 = fc + .15; % beginning of stopband
#     %  pente du filtre > de (-.1dB + 50dB)/.005
#     %  Ordre du filtre
#     [n, wn] = cheb1ord(fc, fc1, .1, 50);
#     if rem(n, 2) == 0 disp('low_pass: ATTENTION, on force n impair ...'); n = n+1; end
#     % Coefficients du filtre
#     [C, D] = cheby1(n, R, wn);
#     % Filtrage
#     sig_out = filtfilt(C, D, sig_in);
# return

def create_lowpass_filter(sampling_rate, cutoff_freq, R=.15):
    fc = cutoff_freq * 2 / sampling_rate
    fc1 = fc + .15
    n, wn = signal.cheb1ord(fc, fc1, .1, 50)
    C, D = signal.cheby1(n, R, wn)
    
    def filt(x):
        y = x.copy()
        nb_dim = x.shape[1]
        for i_dim in range(nb_dim):
            y[:, i_dim] = signal.filtfilt(C, D, x[:, i_dim])
        return y
        
    return filt

ema_lowpass_filter = create_lowpass_filter(500, EMA_LOWPASS_CUTOFF)
files_lowpass_filter = create_lowpass_filter(1000, FILES_LOWPASS_CUTOFF)

In [6]:
def show_dataset_mean(dataset_name):
    dataset = Dataset(dataset_name)
    items_ema = dataset.get_items_data("ema")
    items_name = list(items_ema.keys())
    
    items_mean = []
    for item_name in items_name:
        item_ema = items_ema[item_name]
        item_mean = item_ema.mean(axis=0)
        items_mean.append(item_mean)
    items_mean = np.array(items_mean)
    items_mean = items_mean - items_mean.mean(axis=0)
    items_mean_filtered = files_lowpass_filter(items_mean)
    
    nb_channels = items_mean.shape[1]
    
    plt.figure(figsize=(10, nb_channels), dpi=100)
    for i_channel in range(nb_channels):
        ax = plt.subplot(nb_channels, 1, 1 + i_channel)
        ax.set_ylim(-5, 5)
        ax.plot(items_mean[:, i_channel])
        ax.plot(items_mean_filtered[:, i_channel])
    plt.show()
    
    # plt.figure(figsize=(10, 5), dpi=100)
    # i_channel = 2
    # ax = plt.subplot()
    # # ax.set_ylim(-2, 2)
    # ax.plot(items_mean[:, i_channel])
    # ax.plot(items_mean_filtered[:, i_channel])
    # plt.show()

display(ipw.interactive(show_dataset_mean, dataset_name=datasets_name))

interactive(children=(Dropdown(description='dataset_name', options=('fsew0', 'msak0', 'pb2007'), value='fsew0'…

In [5]:
def show_dataset_item(dataset_name):
    dataset = Dataset(dataset_name)
    items_ema = dataset.get_items_data("ema")
    items_name = list(items_ema.keys())
    
    def show_item_ema(item_name):
        item_ema = items_ema[item_name]
        item_ema_mean = item_ema.mean(axis=0)
        nb_frames, nb_channels = item_ema.shape
        
        xlim = item_ema[:, 0::2].min(), item_ema[:, 0::2].max()
        ylim = item_ema[:, 1::2].min(), item_ema[:, 1::2].max()
        
        item_ema_filtered = ema_lowpass_filter(item_ema)

        plt.figure(figsize=(10, nb_channels), dpi=100)
        for i_channel in range(nb_channels):
            ax = plt.subplot(nb_channels, 1, 1 + i_channel)
            ax.set_ylim(-20, 20)
            ax.plot(item_ema[:, i_channel] - item_ema_mean[i_channel])
            ax.plot(item_ema_filtered[:, i_channel] - item_ema_mean[i_channel])
        plt.show()
        
        def show_ema_frame(i_frame=0):
            ema_frame = item_ema[i_frame]
            ema_frame_filtered = item_ema_filtered[i_frame]
            
            plt.figure()
            ax = plt.subplot(aspect="equal")
            ax.set_xlim(*xlim)
            ax.set_ylim(*ylim)
            ax.scatter(ema_frame[0::2], ema_frame[1::2])
            ax.scatter(ema_frame_filtered[0::2], ema_frame_filtered[1::2])
            plt.show()
            
        display(ipw.interactive(show_ema_frame, i_frame=(0, nb_frames-1)))
    
    display(ipw.interactive(show_item_ema, item_name=items_name))

display(ipw.interactive(show_dataset_item, dataset_name=datasets_name))

interactive(children=(Dropdown(description='dataset_name', options=('fsew0', 'msak0', 'pb2007'), value='fsew0'…